# Literature-Assisted Calibration

- **Goal:** Convert extracted literature facts into parameter suggestions for manual review.
- **Data:** First entry from `reference/goldset/index.json`.
- **Targets:** Preserve extraction quality thresholds (`fact_accuracy ≥ 0.95`, `table_row_recall ≥ 0.9`).


## Steps

1. Run the extraction pipeline on a gold-set PDF.
2. Evaluate the output against the annotated fixture.
3. Generate parameter suggestions using the literature action mapper.
4. Feed accepted suggestions through the analyst console or API for application.


In [ ]:
import json
from pathlib import Path

from mcp_bridge.literature.actions import LiteratureActionMapper
from mcp_bridge.literature.evaluation import evaluate, load_fixture
from mcp_bridge.literature.extractors import (
    HeuristicTextExtractor,
    PdfExtractKitLayoutExtractor,
    SimpleFigureExtractor,
    SimpleTableExtractor,
)
from mcp_bridge.literature.pipeline import LiteratureIngestionPipeline, PipelineDependencies

goldset_root = Path("reference/goldset")
manifest = json.loads((goldset_root / "index.json").read_text(encoding="utf-8"))
entry = manifest["entries"][0]
pdf_path = goldset_root / entry["pdf"]
annotation_path = goldset_root / entry["annotation"]

pipeline = LiteratureIngestionPipeline(
    PipelineDependencies(
        layout_extractor=PdfExtractKitLayoutExtractor(),
        text_extractor=HeuristicTextExtractor(),
        table_extractor=SimpleTableExtractor(),
        figure_extractor=SimpleFigureExtractor(),
    )
)

extraction = pipeline.run(str(pdf_path), source_id=entry["sourceId"])
fixture = load_fixture(annotation_path)
report = evaluate(extraction, fixture)
print(f"fact_accuracy={report.fact_accuracy:.3f}, table_row_recall={report.table_row_recall:.3f}")

mapper = LiteratureActionMapper(simulation_id="usecase-calibration")
suggestions = mapper.map_actions(extraction)
print(f"Generated {len(suggestions)} parameter suggestions")
for suggestion in suggestions[:5]:
    print(f"- {suggestion.tool_name}: {suggestion.summary}")


## Reference Metrics

The gold-set evaluation harness enforces `fact_accuracy ≥ 0.95` and
`table_row_recall ≥ 0.9`. If the printed values fall below those thresholds,
investigate extraction regressions before applying calibration suggestions.
